In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ipyparallel as ipp
print(ipp.version_info)

(8, 6, 1)


In [ ]:
nproc = 4
cluster = ipp.Cluster(engines='mpi', n=nproc, shutdown_atexit=False)
print(cluster)
client = cluster.start_and_connect_sync(activate=True)
view = client[:]
client.ids

<Cluster(cluster_id='1687498951-52eg', profile='default')>
Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.44s/engine]


[0, 1, 2, 3]

In [ ]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1687498951-52eg                  True          4 MPI


In [ ]:
%autopx --block --group-outputs=engine

%autopx enabled


In [ ]:
import numpy as np
from qtmpy.mpi import CommMod, CommManager
from mpi4py.MPI import COMM_WORLD

comm = CommMod(COMM_WORLD)

In [ ]:
with comm:
    arr = np.zeros(10)
    arr[:] = comm.rank
    comm.Allreduce(comm.IN_PLACE, arr, comm.SUM)
    print(arr)

[stdout:1] [6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]


[stdout:2] [6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]


[stdout:0] [6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]


[stdout:3] [6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]


In [ ]:
import time

with comm:
    arr = np.zeros(10)
    with comm.Incl([0, 2]) as c1:
        if not c1.is_null:
            arr[:] = c1.world_rank
            c1.Allreduce(c1.IN_PLACE, arr, c1.SUM)
            time.sleep(2)
            print('a', arr)
    print(comm.rank)
    with comm.Incl([1, 3]) as c2:
        if not c2.is_null:
            arr[:] = c2.world_rank
            c2.Allreduce(c2.IN_PLACE, arr, c2.MIN)
            time.sleep(5)
            print('b', arr)
            

%px:   0%|                                                                                                                                                                                                                                       | 0/4 [00:00<?, ?tasks/s]

[stdout:0] a [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
0


[stdout:2] a [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
2


[stdout:1] 1
b [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


[stdout:3] 3
b [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


%px: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.25s/tasks]


In [ ]:
import time

with comm:
    arr = np.zeros(10)
    with comm.Incl([0, 2], False) as c1:
        if not c1.is_null:
            arr[:] = c1.world_rank
            c1.Allreduce(c1.IN_PLACE, arr, c1.SUM)
            time.sleep(2)
            print('a', arr)
    # print(comm.rank)
    with comm.Incl([1, 3], False) as c2:
        if not c2.is_null:
            arr[:] = c2.world_rank
            c2.Allreduce(c2.IN_PLACE, arr, c2.MIN)
            time.sleep(2)
            print('b', arr)

%px:   0%|                                                                                                                                                                                                                                       | 0/4 [00:00<?, ?tasks/s]

[stdout:3] b [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


[stdout:0] a [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


[stdout:2] a [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


[stdout:1] b [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


%px: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 101.99tasks/s]


In [ ]:
import time

with comm:
    arr = np.zeros(10)
    with comm.Incl([0, 2], sync_with_parent=False) as c1:
        if not c1.is_null:
            arr[:] = c1.world_rank
            c1.Allreduce(c1.IN_PLACE, arr, c1.SUM)
            time.sleep(2)
            print('a', arr)
    # print(comm.rank)
    with comm.Incl([0, 3], sync_with_parent=False) as c2:
        if not c2.is_null:
            arr[:] = c2.world_rank
            c2.Allreduce(c2.IN_PLACE, arr, c2.MAX)
            time.sleep(5)
            print('b', arr)

%px:   0%|                                                                                                                                                                                                                                       | 0/4 [00:00<?, ?tasks/s]

[stdout:2] a [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


[stdout:0] a [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
b [3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


%px:   0%|                                                                                                                                                                                                                                       | 0/4 [00:04<?, ?tasks/s]

[stdout:3] b [3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


%px: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.25s/tasks]


In [ ]:
start = 5
N = 14
print(np.arange(N)[comm.scatter_slice(slice(N))])
print(np.arange(N)[comm.scatter_slice(slice(start, N))])
print(np.arange(N)[comm.scatter_slice(slice(N, start))])

[stdout:1] [4 5 6 7]
[8 9]
[]


[stdout:0] [0 1 2 3]
[5 6 7]
[]


[stdout:3] [11 12 13]
[12 13]
[]


[stdout:2] [ 8  9 10]
[10 11]
[]


In [ ]:
import numpy as np
from qtmpy.mpi import CommMod, CommManager
from mpi4py.MPI import COMM_WORLD

comm = CommMod(COMM_WORLD)


with comm:
    print()
    full_size = comm.size * (comm.size + 1) // 2
    inp_size = comm.size - comm.rank
       
    l_inp_size = comm.size - np.arange(comm.size, dtype='i8')
    
    l_out_size = full_size // comm.size \
        + (full_size % comm.size > np.arange(comm.size, dtype='i8'))
    out_slice = comm.scatter_slice(slice(full_size))

    arr_inp = np.empty((inp_size, ))
    arr_inp[:] = comm.rank
    arr_out = np.empty((l_out_size[comm.rank], ))
    
    inp_proc_map = np.repeat(np.arange(comm.size, dtype='i8'),
                             l_inp_size)
    out_proc_map = np.repeat(np.arange(comm.size, dtype='i8'),
                             l_out_size)
    

    istart = np.sum(l_inp_size[:comm.rank])
    istop = istart + l_inp_size[comm.rank]
    # print(istart, istop)
    sendbuf = np.histogram(out_proc_map[istart:istop],
                           np.arange(comm.size + 1, dtype='i8'))[0]
    recvbuf = np.histogram(inp_proc_map[out_slice], np.arange(comm.size + 1, dtype='i8'))[0]
    
    if comm.rank == 0:
        print(inp_proc_map)
        print(out_proc_map)
    print(sendbuf, recvbuf)
    
    comm().Alltoallv([arr_inp, sendbuf], [arr_out, recvbuf])
    print(arr_out)

[stdout:1] 
[0 2 1 0] [1 2 0 0]
[0. 1. 1.]


[stdout:2] 
[0 0 1 1] [0 1 1 0]
[1. 2.]


[stdout:3] 
[0 0 0 1] [0 0 1 1]
[2. 3.]


[stdout:0] 
[0 0 0 0 1 1 1 2 2 3]
[0 0 0 1 1 1 2 2 3 3]
[3 1 0 0] [3 0 0 0]
[0. 0. 0.]


In [70]:
from mpi4py.MPI import COMM_WORLD
from mpi4py_fft.pencil import Subcomm, Pencil
from mpi4py_fft import PFFT, DistArray


axis = 1
grid = [2, 1, 2]
shape = (18, 18, 18)

subcomm = Subcomm(COMM_WORLD, grid)
arr = DistArray(shape, subcomm, dtype='c16')
pencil_1 = Pencil(subcomm, shape, axis)
pencil_2 = Pencil(subcomm, shape, axis=2)
transfer = 
out = ""
for comm in subcomm:
    out = out + f"{comm.Get_rank()}/{comm.Get_size()} "
print(out)
print(pencil_1.subshape)
print(pencil_2.subshape)

[stdout:3] 1/2 0/1 1/2 
(9, 18, 9)


[stdout:0] 0/2 0/1 0/2 
(9, 18, 9)


[stdout:2] 1/2 0/1 0/2 
(9, 18, 9)


[stdout:1] 0/2 0/1 1/2 
(9, 18, 9)


In [33]:
%autopx --block --group-outputs=engine

%autopx enabled


In [31]:
cluster.restart_engines_sync()

Stopping engine(s): 1687498952
Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
engine set stopped 1687498952: {'exit_code': 1, 'pid': 20731, 'identifier': 'ipengine-1687498951-52eg-1687498952-18123'}


In [32]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1687498951-52eg                  True          4 MPI
